In [1]:
with open("output 2/entity2id.txt" , 'r') as f:
    data = f.read()

In [2]:
data = [i.split("\t") for i in data.split("\n")]

In [3]:
def get_wikipedia_url_from_wikidata_id(wikidata_id, lang='en', debug=False):
    import requests
    from requests import utils

    url = (
        'https://www.wikidata.org/w/api.php'
        '?action=wbgetentities'
        '&props=sitelinks/urls'
        f'&ids={wikidata_id}'
        '&format=json')
    json_response = requests.get(url).json()
    if debug: print(wikidata_id, url, json_response) 

    entities = json_response.get('entities')    
    if entities:
        entity = entities.get(wikidata_id)
        if entity:
            sitelinks = entity.get('sitelinks')
            if sitelinks:
                if lang:
                    # filter only the specified language
                    sitelink = sitelinks.get(f'{lang}wiki')
                    if sitelink:
                        wiki_url = sitelink.get('url')
                        if wiki_url:
                            return requests.utils.unquote(wiki_url)
                else:
                    # return all of the urls
                    wiki_urls = {}
                    for key, sitelink in sitelinks.items():
                        wiki_url = sitelink.get('url')
                        if wiki_url:
                            wiki_urls[key] = requests.utils.unquote(wiki_url)
                    return wiki_urls
    return None 

In [4]:
data[0]

['Q208431', '0']

In [5]:
get_wikipedia_url_from_wikidata_id(data[0][0])

'https://en.wikipedia.org/wiki/Argis'

In [90]:

# import required modules
from bs4 import BeautifulSoup
import requests
 
# get URL
page = requests.get('https://en.wikipedia.org/wiki/Atlanta_Hawks')
 
# scrape webpage
soup = BeautifulSoup(page.content, 'html.parser')
 
list(soup.children)
 
# find all occurrence of p in HTML
# includes HTML tags
print(soup.find_all('p'))
 
print('\n\n')
 
# return only text
# does not include HTML tags
print(soup.find_all('p')[0].get_text())

[<p class="mw-empty-elt">
</p>, <p>The <b>Atlanta Hawks</b> are an American professional <a href="/wiki/Basketball" title="Basketball">basketball</a> team based in <a href="/wiki/Atlanta" title="Atlanta">Atlanta</a>. The Hawks compete in the <a href="/wiki/National_Basketball_Association" title="National Basketball Association">National Basketball Association</a> (NBA) as a member of the league's <a href="/wiki/Eastern_Conference_(NBA)" title="Eastern Conference (NBA)">Eastern Conference</a> <a href="/wiki/Southeast_Division_(NBA)" title="Southeast Division (NBA)">Southeast Division</a>. The team plays its home games at <a href="/wiki/State_Farm_Arena" title="State Farm Arena">State Farm Arena</a>.
</p>, <p>The team's origins can be traced to the establishment of the <b>Buffalo Bisons</b> in 1946 in <a href="/wiki/Buffalo,_New_York" title="Buffalo, New York">Buffalo, New York</a>, a member of the <a href="/wiki/National_Basketball_League_(United_States)" title="National Basketball Leag

In [7]:
!pip install wikipedia

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/anaconda3/envs/jaeyoung/bin/python -m pip install --upgrade pip' command.


In [11]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import wikipedia

In [9]:
import time
from tqdm import tqdm

In [51]:
data[6474]

['Q796293', '6474']

In [52]:
data[6474 + 5728]

['Q753711', '12202']

In [53]:
data[6474 + 5728 + 1766]

['Q590307', '13968']

In [54]:
data[6474 + 5728 + 1766 + 1490]

['Q18154825', '15458']

In [61]:
qnode2description['Q796293']

KeyError: 'Q796293'

In [62]:
qnode2description['Q796293'] = None

In [ ]:
qnode2description = dict()

In [71]:
for qnode, idx in tqdm(data):
    if qnode not in qnode2description:
        url = get_wikipedia_url_from_wikidata_id(qnode)
        count = 0
        # Specify url of the web page
        if url:
            try:
                source = urlopen(url).read()
            except:
                name = url.split("/")[-1]
                name = urllib.parse.quote(name.encode('utf-8'))
                tmp = url.split('/')
                tmp[-1] = name
                url = "/".join(tmp)
                source = urlopen(url).read()

            # Make a soup 
            soup = BeautifulSoup(source,'lxml')
            soup

            # Extract the plain text content from paragraphs
            paras = []
            for paragraph in soup.find_all('p'):
                paras.append(str(paragraph.text))

            # Extract text from paragraph headers
            heads = []
            for head in soup.find_all('span', attrs={'mw-headline'}):
                heads.append(str(head.text))

            # Interleave paragraphs & headers
            text = [val for pair in zip(paras, heads) for val in pair]
            text = ' '.join(text)

            # Drop footnote superscripts in brackets
            text = re.sub(r"\[.*?\]+", '', text)

            # Replace '\n' (a new line) with '' and end the string at $1000.
            text = text.replace('\n', '')[:-11]

            qnode2description[qnode] = text
            time.sleep(2)
        else:
            sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

            connected = False    
            while connected == False:
                try:
                    query = '''
                    PREFIX wd: <http://www.wikidata.org/entity/>
                    PREFIX schema: <http://schema.org/>

                    SELECT ?o
                    WHERE 
                    {
                      wd:''' + qnode + ''' schema:description ?o.
                      FILTER ( lang(?o) = "en" )
                    }
                    '''
                    sparql.setQuery(query)
                    sparql.setReturnFormat(JSON)

                    results = sparql.query().convert()
                    try:
                        res = results["results"]['bindings'][0]['o']['value']
            #             print(qnode + "\t" + res)
                        qnode2description[qnode] = res
                    except:
                        qnode2description[qnode] = None
                    connected = True
                except:
                    print("error", count)
                    count += 1

  0%|▌                                                                                                                                                    | 71/19213 [02:56<13:53:03,  2.61s/it]

error 0


  1%|▊                                                                                                                                                   | 101/19213 [04:05<14:10:18,  2.67s/it]

error 0
error 1
error 2


 10%|██████████████▍                                                                                                                                  | 1907/19213 [1:16:22<12:29:44,  2.60s/it]

...

 80%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████                             | 15359/19213 [10:41:24<2:29:51,  2.33s/it]

error 0


 80%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                            | 15369/19213 [10:41:46<2:43:35,  2.55s/it]

error 0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19213/19213 [10:45:39<00:00,  2.02s/it]


In [73]:
import pickle
with open('qnode2description.data', 'wb') as f:
    pickle.dump(qnode2description, f)

In [74]:
len(qnode2description)

19213

In [97]:
qnode2description[list(qnode2description.keys())[-2]]

'Asociația Club Sportiv CAO 1910 Oradea, commonly known as Club Atletic Oradea (Romanian pronunciation:\xa0), CA Oradea, or simply as CAO, is an amateur Romanian football club based in Oradea, Bihor County. History The team was founded as Nagyváradi Atletikai Club in 1910, when the city was part of Hungary. Over its history, CAO has won one Romanian national title, one Hungarian national title, and one Romanian national cup. It was dissolved in 1963, but was re-established in 2017 and commenced play in the fifth division. Early years (1910–1940) The home ground of "the White and Greens" is the Stadionul Iuliu Bodola, which has a seating capacity of 11,155. The Romanian-Hungarian champion (1940–1963) The game of football had arrived to Nagyvárad (Oradea) in 1902 with inhabitants returning from study or work, abroad or in Budapest. The sport was gaining in popularity, but there was not yet an organized club to represent the town in matches against other towns. Refounding (2017–present) E

In [43]:
urlopen('https://en.wikipedia.org/wiki/Hincapie–Leomo_p/b_BMC')

UnicodeEncodeError: 'ascii' codec can't encode character '\u2013' in position 18: ordinal not in range(128)

In [ ]:
6474

In [ ]:
5727

In [ ]:
1765-1489

In [84]:
import pickle
with open('qnode2description.data', 'rb') as f:
    d = pickle.load(f)

In [85]:
len(d)

19213

In [68]:
d

TypeError: 'dict_keys' object is not subscriptable

In [69]:
d['Q796293']